developed by Antje Loyal, 
modified by Sebastian Gampe

In [1]:
import numpy as np
import sys
import pandas as pd
from PIL import Image
from sklearn.metrics import confusion_matrix
import math
from os import listdir
from os.path import isfile, join, splitext, basename
import matplotlib.cm as cm
import matplotlib.pyplot as plt


# Make sure that caffe is on the python path: 
CAFFE_ROOT = '/home/cnt/caffe-master/python'        # CHANGE THIS LINE TO YOUR Caffe PATH
sys.path.insert(0, CAFFE_ROOT + 'python')
import caffe
import os

In [2]:
# Use GPU or CPU
#caffe.set_mode_cpu()
caffe.set_mode_gpu()

args = sys.argv
PRETRAINED = ""
MODEL_FILE = ""
mean = ""
testDir = ""
labelstxt = ""

In [4]:
# direcory of the used model
modelDir = "/home/cnt/model/"

# get the necessary model and net files
files = [f for f in listdir(modelDir) if isfile(join(modelDir, f))]
for t in files:
    if t.endswith(".caffemodel"):
        PRETRAINED = modelDir + t
    if t.endswith("deploy.prototxt"):
        MODEL_FILE = modelDir + t
    if t.endswith(".binaryproto"):
        meanproto = modelDir + t
    if t.endswith("labels.txt"):
        labelstxt = modelDir + t


# target directory with image to process and ground truth (Portraits) 
testFile = "/home/cnt/data/test.jpg"
groundTruth = "augustus"

# Set path to dircectory with set of images to process (Portraits)
#testDir = "/home/cnt/students/ML/Daten/OCREPortraits/"
#testDir = "/home/cnt/students/ML/Daten/OCRETrainedClasses/"
testDir = "/home/cnt/data/"

In [5]:
# Generating list of all labels
labelsO = open(labelstxt,'r').read().split('\n')
labels = []
for lab in labelsO:
    labels.append(lab.replace(" ","_"))
#labels.replace(" ","_")
if '' in labels:
    del labels[(labels.index(''))]

In [6]:
# convert .binaryproto to .npy
blob = caffe.proto.caffe_pb2.BlobProto()
data = open( meanproto , 'rb' ).read()
blob.ParseFromString(data)
arr = np.array(caffe.io.blobproto_to_array(blob) )
out = arr[0]
np.save( modelDir + "mean.npy" , out )

In [7]:
# Gets the file name and puts together the matching OCRE ID
def getIDFile(image):
    #clear_name = basename(clear_name)
    ocre_id = os.path.splitext(os.path.basename(str(image)))[0]
    ocre_id = "http://numismatics.org/ocre/id/" + ocre_id
    return ocre_id

# Gets the folder name and puts together the matching OCRE ID
def getIDFolder(image):
    ocre_id = os.path.dirname(str(image))
    ocre_id = os.path.basename(os.path.normpath(ocre_id))
    ocre_id = "http://numismatics.org/ocre/id/" + ocre_id
    return ocre_id

In [8]:
# process one image
def processImage(image):
    # loading neural net with trained weights
    net = caffe.Net(MODEL_FILE, PRETRAINED, caffe.TEST)    
    in_shape = net.blobs['data'].data.shape    
    # reshaping data layer to process #images in one batch (bs) (with 3 colors (RGB) and size 244x244)
    net.blobs['data'].reshape(1, 3, 224, 224)
    net.reshape()
    # Set the shape of the input for the transformer
    transformer = caffe.io.Transformer({'data': in_shape})
    # set mean from given mean file (previous converted from the .binaryproto)
    transformer.set_mean('data', np.load(modelDir + 'mean.npy').mean(1).mean(1))
    # order of color channels
    transformer.set_transpose('data', (2,0,1))
    # RGB to BGR
    transformer.set_channel_swap('data', (2,1,0))
    img = Image.open(image)
    # scale all images to 224x224 (VGG16) or 227x227 (AlexNet)
    img = img.resize((224,224), Image.ANTIALIAS)
    #img = img.resize((227,227), Image.ANTIALIAS)
    img = np.array(img).astype(np.float32)
    # Transform the image depending of data layer definition and transformer settings from above
    transformed_image = transformer.preprocess('data', img)
    net.blobs['data'].data[...] = transformed_image

    # Forward pass of the images through the network
    out = net.forward()
    maxClass = labels[out['softmax'][0].argmax()]   
    #print("For picture " + str(image) + " the net predicts: " + str(maxClass) + " and ground truth is: " + groundTruth)
    ocre_id = getIDFolder(image)
    
    # GRAD-CAM implementation
    # Source: https://github.com/gautamMalu/caffe-gradCAM/blob/master/00-classification-gradCAM-Visualization.ipynb
    
    output_prob = out['softmax'][0]
    print ('predicted class is:', output_prob.argmax())

    label_index = output_prob.argmax()
    caffeLabel = np.zeros((1,69))
    caffeLabel[0,label_index] = 1;
  
    vis_layer = 'pool5' # visualization layer
    
    grads=net.backward(diffs=[vis_layer],**{'softmax':caffeLabel})
    #bw=net.backward(diffs=[vis_layer],**{net.outputs[0]: caffeLabel})
    vis_grad= grads['pool5'] # gradients of pool5 layer with respect to output class
    vis_grad = np.squeeze(vis_grad) # removing the extra dimension
    mean_grads = np.mean(vis_grad, axis=(1, 2)) # mean of gradients


    activations = net.blobs[vis_layer].data
    activations = np.squeeze(activations)
    n_nodes = activations.shape[0] # number of nodes
    vis_size = activations.shape[1:] #visualization shape
    vis = np.zeros((7, 7), dtype=np.float32) 


    #generating saliency image
    for i in range(n_nodes):
        activation = activations[i, :, :]
        weight = mean_grads[i]
        weighted_activation = activation*weight
        vis += weighted_activation


    # We select only those activation which has positively contributed in prediction of given class
    vis = np.maximum(vis, 0)   # relu
    vis_img = Image.fromarray(vis, None)
    vis_img = vis_img.resize((224,224),Image.BICUBIC)
    vis_img = vis_img / np.max(vis_img)
    vis_img = Image.fromarray(np.uint8(cm.jet(vis_img) * 255))
    vis_img = vis_img.convert('RGB') # dropping alpha channel
    plt.imshow(vis_img)
    #plt.show()
    image_path = image
    input_image = Image.open(image_path)
    input_image = input_image.resize((224,224))
    input_image = input_image.convert('RGB')

    #print(vis_img.size, input_i)
    heat_map = Image.blend(input_image, vis_img, 0.3)
    plt.imshow(heat_map)
    plt.axis('off')
    #plt.show()
    return (ocre_id, maxClass, heat_map)

In [ ]:
ocre_id, maxClass, image = processImage(testFile)
image.save('/home/cnt/data/heatmaps/' + maxClass + '_grad.jpg')
print(ocre_id + ' ' + maxClass)

In [8]:
# List all image files (.jpg or .png or .tif) in subfolders of dir
def list_files(dir):
    r = []                                                                                                                                                                                                      
    for dirpath, subdir, files in os.walk(dir):
        for file in files:
            if (file.endswith(".jpg")) or (file.endswith(".png")) or (file.endswith(".tif")): 
                r.append(os.path.join(dirpath, file))                                                                         
    return r 

In [11]:
# use single image mode to create heatmaps for all images in a certain directory
testDirHeat = "/home/cnt/data/augustus/"
save_path = '/home/cnt/data/heatmaps/01'
if not os.path.exists(save_path):
    os.makedirs(path)    
files = list_files(testDirHeat)
for testFile in files:
    ocre_id, maxClass, image = processImage(testFile)
    pathWords = testFile.split('/')
    image.save(path + maxClass + "-" + (pathWords[len(pathWords)-4].replace('.', '')) + "-" + (pathWords[len(pathWords)-3].replace('.', '')) + "-" + (pathWords[len(pathWords)-2].replace('.', '')) + '.jpg')

predicted class is: 14


In [ ]:
# Use the trained  model on images in single image mode and write the results to a pandas DataFrame 
result_dict = {}
result_dict["DesignID"] = []
result_dict["Results"] = []

moreImages = list_files(testDir)
for image in moreImages:
    result = processImage(image)
    result_dict["DesignID"].append(result[0])
    result_dict["Results"].append(result[1])



erg = pd.DataFrame({"DesignID" : result_dict["DesignID"],
                   "y_predict" : result_dict["Results"]})

erg.head()